In [11]:
import numpy as np 
import pandas as pd
from itertools import combinations
import matplotlib.pyplot as plt
import random
import os 
delta =0.1
number_of_particles=1000
time_to_observe=500

In [12]:
class Particle:

    def __init__(self, vel):
        self.vel = vel
        # vel is a numpy array representing velocity
    def display_info (self):
        print('Velocity: %.2f, %.2f' % (self.vel[0], self.vel[1]))        
    def inelastic_collide_particle (self, particle2):
        angle_of_attack = 2*np.pi* random.random()
        v1=self.vel
        v2=particle2.vel
        pos_diff=np.array([np.cos(angle_of_attack),np.sin(angle_of_attack)])
        # pos_diff is a random unit vector
        value=-1*np.dot(pos_diff,v1-v2)
        if (value**2-4*(1-gamma)*(0.5*np.dot(v1,v1)+0.5*np.dot(v2,v2))<0):
            #print('This collision cannot happen')
            return 
        else:
            if (value>0):
                new_value=0.5*value+0.5*np.sqrt(value**2-4*(1-gamma)*(0.5*np.dot(v1,v1)+0.5*np.dot(v2,v2)))
            elif(value==0):
                new_value=0
            else:
                new_value=0.5*value-0.5*np.sqrt(value**2-4*(1-gamma)*(0.5*np.dot(v1,v1)+0.5*np.dot(v2,v2)))
            new_v1=v1+new_value*pos_diff
            new_v2=v2-new_value*pos_diff
            self.vel=new_v1
            particle2.vel=new_v2
        #print('The gamma of this collision is:')
        #print((np.dot(new_v1,new_v1)+np.dot(new_v2,new_v2))/(np.dot(v1,v1)+np.dot(v2,v2)))

In [13]:
class Node:

    def __init__ (self, time, pos, particles):
        self.time = time
        self.pos  = pos
        self.particles = particles

        # particles is an numpy array of Particles
    def display_info (self):
        print('Time: %.2f' % self.time)
        print('Position: %.2f, %.2f' % (self.pos[0], self.pos[1]))
        print('The particles within this node: ')
        for i in self.particles:
            print('++++++++++++++++++++++++++++++++++++++++++++')
            i.display_info()
            print('++++++++++++++++++++++++++++++++++++++++++++')        
    def collide_node (self,collision_rate):
        # This function takes in the number of pairs of particles to collide
        self.time=self.time+ delta
        index=list(range(len(self.particles)))
        number_of_pairs=int(len(self.particles)*collision_rate/2)
        pairs=list(range(number_of_pairs)) # initialize pairs to be of equal length
        # Record the pairs of indexes of particles to collide in the list pairs
        for i in range(number_of_pairs):
            new_pair=list(random.sample(index,2))
            pairs[i]=new_pair
            index=list(set(index)-set(new_pair))
        for i in pairs:
            self.particles[i[0]].inelastic_collide_particle(self.particles[i[1]])       
    def hist (self,number, direc):
        # Plot a 2D histograph for the distribution of velocity at this node
        length=len(self.particles)
        X=list(range(length))
        Y=list(range(length))
        for i in X:
            X[i]=self.particles[i].vel[0]
            Y[i]=self.particles[i].vel[1]
        plt.hist2d(X,Y,bins=(50,50))
        plt.title("2D Histogram #"+str(number))
        plt.savefig(direc +'simu_hist'+str(number)+".png")    
    def mean_speed(self):
        speed_sum=0
        for i in self.particles:
            speed_sum+=np.linalg.norm(i.vel)
        return speed_sum/len(self.particles)

    def mean_velocity(self):
        x=0
        y=0
        for i in self.particles:
            x+=i.vel[0]
            y+=i.vel[1]
        return np.array([x,y])/len(self.particles)

    def total_energy(self):
        total_energy=0
        for i in self.particles:
            total_energy+= 0.5*np.dot(i.vel,i.vel)   
        return total_energy/len(self.particles)

    def kinetic_energy (self):
        mean_vel=self.mean_velocity()
        return 0.5 * np.dot(mean_vel,mean_vel) 

    def potential_energy(self):
        return self.total_energy()-self.kinetic_energy()

In [14]:
for i in [0.25,0.5,0.75]:   # collision rate  
    collision_rate=i
    for j in [0.4,0.6,0.8,1]: # gamma  
        gamma=j
        name= str(number_of_particles)+ 'particles'+str(i)+'collision rate'+str(j)+ 'gamma'
        directory='/Users/lm4307/Desktop/long_Gaussian_inelastic/'+ name + '/'    
        parent_path = r'C:\Users\lm4307\Desktop\long_Gaussian_inelastic' 
        new_path=parent_path + '\\'+ name
        if not os.path.exists(new_path):
            os.makedirs(new_path)
        #Generate a new node named Node1, whose position is [1,1]
        Node1=Node(0,[1,1],[])
        #Generate an array of velocities following the 2D Gaussian Distribution with mean of x and y 1, covariace matrix being the identity matrix
        velocities=np.random.multivariate_normal([1,1],[[1,0],[0,1]],number_of_particles)
        #Put all the particles into Node1
        for ii in range(number_of_particles):
            new_particle= Particle(velocities[ii])
            lst=Node1.particles
            lst.append(new_particle)
            Node1.particles=lst
            
        mean_speed_list=list(range(time_to_observe))
        total_energy_list=list(range(time_to_observe))
        kinetic_energy_list=list(range(time_to_observe))
        potential_energy_list=list(range(time_to_observe))
        
        #implement inelastic collision
        for k in range (time_to_observe):
            Node1.collide_node(collision_rate)
            Node1.hist(k,directory)
            mean_speed_list[k]=Node1.mean_speed()
            #print('task' + str(i) + 'finished---1')
            total_energy_list[k]=Node1.total_energy()
            #print('task'+ str(i) +'finished---3')
            kinetic_energy_list[k]=Node1.kinetic_energy()
            #print('task'+ str(i) +'finished---4')
            potential_energy_list[k]=Node1.potential_energy()
            #print('task'+ str(i) +'finished---5')
            
        #save these plots
        plt.close()
        plt.plot(mean_speed_list)
        plt.title('mean speed')
        plt.savefig(directory+'mean_speed.png')
        plt.close()
        plt.plot(kinetic_energy_list)
        plt.title('kinetic energy')
        plt.savefig(directory+'kinetic_energy.png')
        plt.close()
        plt.plot(total_energy_list)
        plt.title('total energy')
        plt.savefig(directory+'total_energy.png')
        plt.close()
        plt.plot(potential_energy_list)
        plt.title('potential energy')
        plt.savefig(directory+'potential_energy.png')
        plt.close()